In [ ]:
import networkx as nx
import random
import matplotlib.pyplot as plt

### Part One
- Defining a network topology
- Randomly assigning colours to each node 
- Running experiments to see if correct colouring of the graph can be reached.
- Counting the number of conflicts over time

In [ ]:
n = 60
k = 4
p = 0.7

G = nx.random_graphs.connected_watts_strogatz_graph(n, k, p)


In [ ]:
nx.draw(G)

In [ ]:
A = nx.adjacency_matrix(G)
print(A.todense())

In [ ]:
collisions_c = []

colours = []

for i in range(3, 12):
    colours.append({'num': i, 'max_collisions': 0, 'min_collisions': 50})

def check_colouring(G, num_colours, collision, max_collisions, min_collisions):
    for node in G:
        for n in G.neighbors(node):
            if G.nodes[node]['colour'] == G.nodes[n]['colour'] and node > n:
                G.nodes[node]['colour'] = random.randint(0, num_colours-1)
                collision = collision + 1


    if collision > max_collisions:
        max_collisions = collision

    if collision < min_collisions:
        min_collisions = collision
    
    return collision, max_collisions, min_collisions

for i in range(len(colours)):
    collisions_c.append([])
    for j in range(G.order()):        
        G.nodes[j]['colour'] = random.randint(0, colours[i]['num']-1)
    for j in range(10):
        collisions_c[i].append(0)
        collisions_c[i][j], colours[i]['max_collisions'], colours[i]['min_collisions'] = check_colouring(G, colours[i]['num'], collisions_c[i][j], colours[i]['max_collisions'], colours[i]['min_collisions'])
    print(f"Collisions over time for {colours[i]['num']} colours: {collisions_c[i]}")
    print(f"Max collisions: {colours[i]['max_collisions']} for number of colours: {colours[i]['num']}")
    print(f"Min collisions: {colours[i]['min_collisions']} for number of colours: {colours[i]['num']}")


In [ ]:
import csv

with open('collisions.csv', mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Number of colours', 'Collisions over time', 'Max collisions', 'Min collisions'])
    for i in range(len(colours)):
        writer.writerow([colours[i]['num'], collisions_c[i], colours[i]['max_collisions'], colours[i]['min_collisions']])